# Two-Particle Self-Consistent approach (TPSC) tutorial

This tutorial is done in four steps

1. You will first learn how to manipulate multivariable Green's functions . You will also
   convince yourself that for nearest-neighbor non-interacting models, the Fermi surface has perfect nesting
   (TPSC-1)

2. You will compute the Lindhard function for the non-interacting susceptibility (TPSC-2)

3. You will then compute the RPA approximation to check the divergence at ($\pi,\pi)$ (TPSC-3)

4. Renormalized spin and charge vertices in TPSC are computed. (TPSC-4)
   The spin susceptibility is computed to show that it does not diverge at finite temperature.

5. A challenging exercice with the self-energy is left at the end if you have time. (TPSC-5)

## Lattice Green function

In this notebook, we will first manipulate a bit the Green function on a lattice, 

\begin{equation}
G_0(\mathbf{k},\omega)=\frac{1}{\omega + i\eta - \epsilon(\mathbf{k})}
\end{equation}
with, in units where lattice spacing $a=1$,
\begin{equation}
\epsilon(\mathbf{k})=-2t(\cos{k_x}+\cos{k_y})
\end{equation}
$\mathbf{k}$ is a vector in the Brillouin zone.

In [ ]:
%matplotlib inline
from pytriqs.plot.mpl_interface import plt
import numpy as np
from math import cos, pi
from pytriqs.lattice import BravaisLattice, BrillouinZone
from pytriqs.gf import MeshBrillouinZone, MeshImFreq, Gf, MeshProduct, Idx

In [ ]:
beta = 1./0.4
t = 1.0 

n_k = 128
n_w = 128

## A mesh on a Brillouin Zone

We first need a simple Bravais Lattice in 2 dimensions,  its corresponding Brillouin Zone and a mesh on it.

In [ ]:
# Two unit vectors in R3
BL = BravaisLattice([(1, 0, 0), (0, 1, 0)])

# The Brillouin zone
BZ = BrillouinZone(BL)

The Green function $$ G(k, i\omega_n) $$
is on a cartesian product mesh : $ (k \times i\omega_n) \rightarrow {\mathcal{C}}$

In [ ]:
# Construct the Green function

# A regular mesh on the BZ with 30 x 30 points
kmesh = MeshBrillouinZone(BZ, n_k = n_k)

# Frequency mesh
wmesh = MeshImFreq(beta=beta, S='Fermion', n_max=n_w)

g0 = Gf(mesh = MeshProduct(kmesh, wmesh),target_shape = [])

print g0, g0.rank, g0.data.shape

In [ ]:
iGamma = 0.01 * 1j

def eps(k):
    return -2 * t* (cos(k[0]) + cos(k[1]))

# NB : loop is a bit slow in python ...
for k in g0.mesh[0]:
    for w in g0.mesh[1]:
        g0[k,w] = 1/(w - eps(k) + iGamma)

## Save Green function for later use

In [ ]:
from pytriqs.archive import HDFArchive
with HDFArchive("tpsc.h5") as R:
    R['g0_kw'] = g0

## For nearest-neighbor model, the Fermi surface is nested

### Plot of the momentum distribution curve at the Fermi level

The Fermi surface is nested. 

    What do we mean by that?
    What is the nesting vector?

In [ ]:
# NOW HERE we plot G(k, om)
# We show how to slice for fixed k, fixed omega
# and make a SIMPLE plot

# Fix FIX the evaluator of the GF to be periodic
# We use it on a grid on BZ ...

k = np.linspace(-np.pi, np.pi, kmesh.linear_dims[0]+1, endpoint=True)
kx, ky = np.meshgrid(k, k)

spectral = lambda kx, ky: g0([kx,ky,0],0).imag
fs = lambda kx, ky: (1/g0([kx,ky,0],0)).real

plt.figure(figsize=(7,7))
plt.pcolor(kx, ky, np.vectorize(spectral)(kx,ky))
plt.contour(kx, ky, np.vectorize(fs)(kx,ky), levels=[0], colors='white')
plt.axes().set_aspect('equal')

plt.xticks([-np.pi, 0, np.pi],[r"$-\pi$", r"0", r"$\pi$"])    
plt.yticks([-np.pi, 0, np.pi],[r"$-\pi$", r"0", r"$\pi$"])
plt.xlabel(r"$k_x$")
plt.ylabel(r"$k_y$")
plt.title("Momentum distribution curve (MDC) at the Fermi level")

## Density

It would be nice to show the density along a diagonal in $k$ to see the signature of the Fermi surface.

In [ ]:
d = lambda k: g0[Idx(k,k,0),:].density().real
kr = range(15)

plt.plot(kr, np.vectorize(d)(kr), '-o')

## Simple tests (will be removed)

In [ ]:
k = np.linspace(0, 2*np.pi, n_k+1, endpoint=True)
check = lambda kx, ky: g0([kx,ky,0],0).imag

# Should be 0
print g0.data.reshape(n_k, n_k, 2*n_w)[:,0,n_w].imag - np.vectorize(check)(k,0)[0:n_k]

mf = lambda n : (2*n+1)*pi/beta*1j
for k in g0.mesh[0]:
    assert abs(g0(k.value, 0) - 1/(mf(0) - eps(k) + iGamma))< 1.e-10